## LangChain

### What is LangChain?
LangChain is an open-source framework designed to facilitate the development of applications that leverage large language models (LLMs) like OpenAI’s GPT, Hugging Face models, or similar technologies. Its primary goal is to make it easier to build applications that use LLMs in more structured, robust, and context-aware ways. LangChain focuses on combining LLMs with external data, chaining multiple components together for more complex workflows, and managing contextual information effectively.

To find out more about the repository devoted to open-source LangChain belongings and libraries click [here](https://github.com/langchain-ai/langchain).

Below we will have a quick review of how to create a client, make a request and collect the corresponding response:

In [2]:
import openai
import langchain
from langchain.llms import OpenAI

In [5]:
# Generate API Key
my_openaiapi_key = "Add your own key here!"

# openai.api_key = my_openaiapi_key
client = OpenAI(openai_api_key=my_openaiapi_key)

In [4]:
# Come up with a prompt
prompt = "Can you tell how many automobile companies there are in the world?"

In [22]:
response = client.predict(prompt)

In [29]:
response, response.strip()

('\n\nAs of 2021, there are approximately 98 automobile companies in the world.',
 'As of 2021, there are approximately 98 automobile companies in the world.')

`strip()` removes all whitespace at the start and end, e.g., spaces, tabs, newlines and carriage returns.

In [30]:
print(response)



As of 2021, there are approximately 98 automobile companies in the world.


### List of Content
- How to use OpenAI via LangChain
- Prompt Templating
- Agent
- Chains
- Document Loader
- Memory

### How can we utilize OpenAI through LangChain?
#### Why OpenAI API? Why not OpenAI API?
Why OpenAI API?
- **State-of-the-Art Models:** Provides access to GPT (e.g., GPT-4), which is among the most advanced language models available.
- **Ease of Use:** Simple and intuitive API with robust documentation for seamless integration.
- **Versatility:** Supports a wide range of applications, such as text generation, summarization, code completion, translation, and more.
- **Scalability:** Handles workloads of various sizes, suitable for startups, enterprises, or research projects.
- **No Need for Infrastructure:** Eliminates the need to manage hardware or train your own models.

Why Not OpenAI API?
- **Cost:** Pricing may be prohibitive for large-scale usage or for those with tight budgets.
- **Data Privacy Concerns:** User data is sent to OpenAI’s servers, which may not align with strict data privacy requirements in sensitive industries.
- **Dependence on External Service:** Reliance on an external API could lead to downtime or latency issues.
- **Customization Limitations:** While powerful, the models are pre-trained and may not be fine-tuned for niche use cases without additional effort.
- **Alternative Models:** Other open-source models like Hugging Face or LLaMA offer free, customizable, and local deployment options, which can be more suitable for certain projects.

#### What makes us use LangChain?
LangChain is used because it simplifies building end-to-end applications that combine LLMs (Large Language Models) with other tools, enabling more dynamic and intelligent workflows.

LangChain is used for simplifying the development of applications that integrate language models with external data sources, APIs, and workflows. It allows developers to chain together various components like language models, databases, and web scraping tools, enabling more complex interactions and tasks like document querying, summarization, and multi-step reasoning. LangChain also supports a modular approach for integrating multiple sources of information, offering features like prompt management, model wrappers, and memory management, which make it ideal for building scalable, customizable, and efficient applications in areas like chatbots, document analysis, and knowledge extraction.

### Prompt Template

In order to dig deeper into the world of LangChain, let's firstly discuss what **Prompt Templates** (`langchain.prompts.prompt.PromptTemplate()`) are. A prompt template is a scaffold or blueprint that guides the AI in generating a response. It helps to translate user input and parameters into instructions for a language model.

In [5]:
from langchain.prompts import PromptTemplate

prompt_template = PromptTemplate(
    input_variable=["product"],
    template="Can you tell how many {product} companies there are in the world?"
)

In [43]:
prompt_template2 = PromptTemplate(
    input_variable=["product"],
    template="Can you tell approximately how many {product} companies there are in the Italy?"
)

In [44]:
# define a few prompts based on the template
prompt1 = prompt_template.format(product="mobile phone")
prompt2 = prompt_template2.format(product="start-up")

In [36]:
client.predict(prompt1).strip()

'It is difficult to determine the exact number of mobile phone companies in the world as new companies are constantly emerging and existing companies may merge or go out of business. However, as of 2021, there are over 100 major mobile phone companies operating globally. Some of the largest companies include Samsung, Apple, Huawei, Xiaomi, and Oppo.'

**REMEMBER** that you can also pass the argument `max_tokens` to limit the output tokens!

In [47]:
client.predict(prompt2, max_tokens=50).strip()

'It is difficult to provide an exact number as the number of start-up companies in Italy is constantly changing. However, according to a report by Startup Genome, there were approximately 10,000 active start-ups in Italy in 2019. This'

#### What is strange about the response produced above?
Oh, yes! You're right. The response is incomplete according to the limitation we have applied to the number of output tokens.

An alternative way to create a prompt template is `PromptTemplate.from_template()` not passing the input variable explicitly. Let's see how it works below:

In [52]:
prompt_template3 = PromptTemplate.from_template("What is a synonym to the word {word} that you can come up with?")

In [59]:
prompt3=prompt_template3.format(word="habit")

In [60]:
client.predict(prompt3).strip()

'Routine'

### Agents
An agent in an LLM application is a mechanism that enables the language model to **interact with external tools, APIs, or environments** dynamically. It interprets user inputs, decides which actions to take (e.g., calling APIs, retrieving documents, or running computations), and integrates the results into coherent responses. This allows for complex, task-oriented workflows, such as executing code, answering questions from databases, or performing **real-world** and in some cases **real-time** tasks, making the application more interactive and functional.

**SerpAPI** is an API that provides search engine results. An agent, in this context, would be the system or mechanism (often part of an LLM application) that decides to query the SerpAPI, processes the returned results, and incorporates them into its output. For a Google search engine, the agent would typically be the logic or program (like LangChain's tools or OpenAI's function calling) that triggers a search using SerpAPI and handles the results effectively. SerpAPI does not retrieve information solely through Google. While it is well-known for integrating with Google Search, SerpAPI supports multiple search engines and platforms, including Bing, DuckDuckGo, Yahoo, and others, as well as specialized services like Google Maps, Google Images, and more. This versatility allows it to retrieve information from various sources, not just Google.

`google-search-engine` is a package meant to scrape and parse search results from Google, Bing, Baidu, Yandex, Yahoo, Home Depot, eBay and more, using SerpApi. The following services are provided: Search API. Search Archive API.

In [2]:
!pip install google-search-results

In [7]:
# Generate SerpAPI key
my_serpapi_key = "Add your own key here!"

In [10]:
from langchain.llms import OpenAI
from langchain.agents import initialize_agent, AgentType, load_tools

# Load tools - they allow agents to interact with various resources and services like APIs, databases, file systems, etc.
tools = load_tools(["serpapi"], serpapi_api_key=my_serpapi_key, llm=client)

# Load an agent executor given tools and LLM
agent = initialize_agent(tools, client, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True)

In [9]:
agent.run("Who won Open Australia in 2021?")

C:\Users\ASUS K513EQ\AppData\Local\Temp\ipykernel_16424\3405584166.py:1: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  agent.run("Who won Open Australia in 2021?")




> Entering new AgentExecutor chain...
 You should always think about what to do
Action: Search
Action Input: "Open Australia 2021 winner"
Observation: {'title': 'Australian Open', 'date': 'Jan 10–Feb 22, 2021', 'tables': {'title': "Women's Singles", 'games': [{'date': 'Sat Feb 20', 'stage': 'Final', 'location': 'Rod Laver Arena', 'status': 'Final', 'video_highlights': {'link': 'https://ausopen.com/match/2021-jennifer-brady-vs-naomi-osaka-ws701#!media?g=video-6233932784001', 'thumbnail': 'https://serpapi.com/searches/6799187ed959597baef45f34/images/7306be2960b5a85313a35411f99a753b03c2f8dfba89a0eb2d80884dd8c53e9755b2e90be0fc83fc930720e710985f3183c4eae1af9c89fca6fbbea828fa4dd9.jpeg', 'duration': '2:00'}, 'players': [{'name': 'J. Brady', 'ranking': '22', 'thumbnail': 'https://serpapi.com/searches/6799187ed959597baef45f34/images/7306be2960b5a85313a35411f99a753b03c2f8dfba89a0eb7f7addea4be0791551988c858d272ac0976428454f9e5288f74410045e50db24.png', 'sets': {'set-1': '4', 'set-2': '3'}}, {'na

'Naomi Osaka'

Now let's try a different resource, for instance, ArXiv API (also Wikipedia), for accessing scientific papers and metadata from the ArXiv repository. Firstly, you have to install `arxiv`.

In [11]:
# !pip install arxiv

**Note** that one can pass several tools or resources to `load_tools()` as shown below:

In [18]:
tools = load_tools(["arxiv", "serpapi"], serpapi_api_key=my_serpapi_key, llm=client)

agent = initialize_agent(tools, client, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True)

In [8]:
agent.run("Could you please provide me with the most recent papers on breast cancer exploiting vision transformers since 2024?")

C:\Users\ASUS K513EQ\AppData\Local\Temp\ipykernel_18140\2575220378.py:1: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  agent.run("Could you please provide me with the most recent papers on breast cancer exploiting vision transformers since 2024?")




> Entering new AgentExecutor chain...
 I should search on arxiv for papers related to breast cancer and vision transformers since 2024.
Action: arxiv
Action Input: "breast cancer vision transformers since 2024"
Observation: Published: 2024-06-29
Title: Development of an interactive GUI using MATLAB for the detection of type and stage of Breast Tumor
Authors: Poulmi Banerjee, Satadal Saha
Summary: Breast cancer is described as one of the most common types of cancer which
has been diagnosed mainly in women. When compared in the ratio of male to
female, it has been duly found that the prone of having breast cancer is more
in females than males. Breast lumps are classified mainly into two groups
namely: cancerous and non-cancerous. When we say that the lump in the breast is
cancerous, it means that it can spread via lobules, ducts, areola, stroma to
various organs of the body. On the other hand, non-cancerous breast lumps are
less harmful but it should be monitored under proper diagnosis

'The most recent papers on breast cancer exploiting vision transformers since 2024 are "Development of an interactive GUI using MATLAB for the detection of type and stage of Breast Tumor," "Deep Transfer Learning for Breast Cancer Classification," and "Deep Learning in Breast Cancer Imaging: A Decade of Progress and Future Directions."'

### Chains
According to LangChain official documentation, chains refer to sequences of calls - whether to an LLM, a tool, or a data preprocessing step. The primary supported way to do this is with [LCEL]("https://python.langchain.com/v0.1/docs/expression_language/") (LangChain Expression Language).

In [17]:
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

prompt_template4 = PromptTemplate.from_template("In a word, which country is famous for its {product}?")
# prompt4=prompt_template4.format(product="chocolate") # This step is not needed, automatically taken care of!
# type(prompt_template4), type(prompt4)
# print(prompt_template4)
# print(prompt4)

In [49]:
# Run query against any LLM
chain = LLMChain(llm=client, prompt=prompt_template4)
chain.run("chocolate").strip()

'Switzerland.'

#### Chains in LangChain are not necessarily about chaining multiple LLMs but about combining LLMs with other tools and processes. For example:

- **Simple Chain:** Taking user input, formatting it into a prompt, querying an LLM, and returning the response.
- **Complex Chain:** Integrating an LLM to answer a query, using its output to fetch data from an external API or database, and summarizing the results using the LLM again.

In [66]:
# Simple Chain

prompt_template_meal = PromptTemplate(
    input_variables=["meal"],
    template="What dish do you suggest I should have for {meal} today?"
)


meal_request = LLMChain(llm=client, prompt=prompt_template_meal)

In [75]:
prompt_template_recepie = PromptTemplate(
    input_variables=["dish"],
    template="Provide me with the ingredients for {dish}."
)

recepie_request = LLMChain(llm=client, prompt=prompt_template_recepie)

In [16]:
from langchain.chains import SimpleSequentialChain

In [77]:
# Form a chain associating the two requests
simple_chain = SimpleSequentialChain(chains=[meal_request, recepie_request])

Below the command `simple_chain.run("lunch")` runs a predefined sequence of steps (the chain) using "hunger" as the input, and returns the final output after completing all the steps in the chain.

In [78]:
# chain.invoke(dish_request)
suggested_dish = simple_chain.run("lunch")

In [79]:
print(suggested_dish)

 Here are some suggested ingredients:

For the grilled chicken salad:
- Boneless, skinless chicken breasts
- Mixed greens (such as spinach, arugula, or spring mix)
- Cherry tomatoes
- Cucumber
- Bell pepper
- Red onion
- Olive oil
- Balsamic vinegar
- Salt and pepper

For the vegetarian stir-fry:
- Assorted vegetables (such as broccoli, carrots, mushrooms, bell peppers, onions, and snow peas)
- Tofu or tempeh (optional)
- Soy sauce
- Garlic
- Ginger
- Brown sugar
- Rice or noodles

For the classic sandwich:
- Bread (such as whole wheat, sourdough, or ciabatta)
- Deli meat (such as turkey, ham, or roast beef)
- Cheese (such as cheddar, Swiss, or provolone)
- Lettuce
- Tomato
- Avocado
- Bacon
- Mayo or mustard (optional)
- Chips or soup (optional side dish)


#### Can we also retrieve the intermediate answers?
There are ways to access the intermediate responses of a chain. One could be through setting `output_key` and `output_variables` in the request and chain respectively.

In [85]:
# Step 1: Define a prompt for step 1
step_1_prompt = PromptTemplate(
    input_variables=["input_text"],
    template="Extract a name from the following text: {input_text}"
)

step_1_request = LLMChain(llm=client, prompt=step_1_prompt, output_key="name")

# Step 2: Define a prompt for step 2
step_2_prompt = PromptTemplate(
    input_variables=["name"],
    template="Does {name} start with letter M?"
)

step_2_request = LLMChain(llm=client, prompt=step_2_prompt, output_key="answer")

In [92]:
from langchain.chains import SequentialChain

# Create the chain
chain = SequentialChain(
    chains=[
        step_1_request,  # Step 1
        step_2_request   # Step 2
    ],
    input_variables=["input_text"],
    # return_intermediate_steps=True  # Enable intermediate steps
    output_variables=["name", "answer"]
)

In [94]:
answers = chain({"input_text": "He is Milad"})

In [95]:
print(answers)

{'input_text': 'He is Milad', 'name': '\n\nMilad', 'answer': '\n\nYes'}


In LangChain, there is also this **Document Loader** which is a component responsible for loading and processing data from various sources into a format that can be used by the system, typically as text or structured documents. These sources can range from local files (like PDFs, CSVs, or Word documents) to online resources (like web pages, APIs, or databases). In order to be able to implement a document loader, you will have to have **pypdf** library installed and available in your project environment via the command `!pip install pypdf`.

In [99]:
from langchain.document_loaders import PyPDFLoader

loader = PyPDFLoader(r"C:\Users\ASUS K513EQ\Downloads\Mammography Reporting.pdf")

In [101]:
pages = loader.load_and_split()
print(pages)

[Document(metadata={'source': 'C:\\Users\\ASUS K513EQ\\Downloads\\Mammography Reporting.pdf', 'page': 0, 'page_label': '1'}, page_content='ACR BI-RADS® ATLAS — MAMMOGRAPHY\nAmerican College of Radiology 121\nMAMMOGRAPHY\nII. REPORTING SYSTEM'), Document(metadata={'source': 'C:\\Users\\ASUS K513EQ\\Downloads\\Mammography Reporting.pdf', 'page': 1, 'page_label': '2'}, page_content='2013\n122 American College of Radiology  \nMAMMOGRAPHY'), Document(metadata={'source': 'C:\\Users\\ASUS K513EQ\\Downloads\\Mammography Reporting.pdf', 'page': 2, 'page_label': '3'}, page_content='ACR BI-RADS® ATLAS — MAMMOGRAPHY\nAmerican College of Radiology 123\nMAMMOGRAPHY\nA. REPORT ORGANIZATION ( Guidance chapter, see page 147)\nThe reporting system should be concise and organized using the following structure. A statement \nindicating that the current examination has been compared to previous examination(s) should be \nincluded (specify date[s]). If this is not included, it should be assumed that no comp

#### Why are document loaders important?
Document loaders play a crucial role in automating workflows, as they enable LangChain to interact seamlessly with various data sources. For instance:

- In a legal document analysis application, a loader can extract and preprocess contracts from a folder of PDFs.
- For a question-answering system, a loader can retrieve articles from the web and split them into chunks for embedding and retrieval.

### Memory
LLMs like ChatGPT simulate memory within a single conversation by maintaining a context window that stores previous prompts and responses. This allows the model to generate coherent replies by referring back to earlier parts of the conversation. However, this "memory" is session-specific and limited by the model's context length (e.g., GPT-4 has a maximum token limit).

Outside a session, true memory doesn't exist unless explicitly implemented (e.g., external memory or logging via APIs). This differs from human-like memory, as it’s limited to processing textual patterns and lacks long-term storage.

In [15]:
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.llms import OpenAI

In [23]:
prompt_template_lang = PromptTemplate(
    input_variables = ["language"],
    template = "What is a country in Europe that speaks {language}?"
)

In [54]:
# Create client
client = OpenAI(openai_api_key=my_openaiapi_key, temperature=1.0)

# Create your language request
country_request = LLMChain(llm=client, prompt=prompt_template_lang)

In [32]:
print(country_request.run("English"))



Ireland is a country in Europe where English is the primary language spoken. It is the only country in Europe with English as its official language. Other countries in Europe may have English as a secondary or widely spoken language, but it is not their official language.


**ConversationBufferMemory** is a concept used in frameworks like LangChain to mimic memory for LLMs during a conversation. It involves storing the history of the dialogue (prompts and responses) in a buffer and providing this history as part of the input to the model in each turn. This ensures the model has context from the entire conversation.

While LLMs themselves don’t have inherent long-term memory, ConversationBufferMemory helps manage and feed the conversation history systematically, enabling the model to generate coherent responses by "remembering" earlier parts of the dialogue. This approach is especially useful when building conversational agents.

In [36]:
from langchain.memory import ConversationBufferMemory

# Define the memory and the LLM
memory = ConversationBufferMemory()

# Create a conversation chain with memory
country_request = LLMChain(llm=client, prompt=prompt_template_lang, memory=memory)

Now let's trigger a chat between the *User* and *AI* passing several keywords to form a conversational thread:

In [37]:
country_request.run("Dutch")

'\n\nThe Netherlands is a country in Europe that speaks Dutch.'

In [38]:
country_request.run("German")

'\n\nGermany is the main country in Europe that speaks German. Austria and Switzerland also have significant populations who speak German, and it is recognized as an official language in Liechtenstein, Luxembourg, and Belgium. Parts of Italy, France, and Poland also have German-speaking communities.'

In [39]:
country_request.run("Irish")

'\n\nIreland is the only country in Europe where Irish (Gaeilge) is recognized as an official language and is widely spoken.'

In [49]:
# Access the memory
country_request.memory

ConversationBufferMemory(chat_memory=InMemoryChatMessageHistory(messages=[HumanMessage(content='Dutch', additional_kwargs={}, response_metadata={}), AIMessage(content='\n\nThe Netherlands is a country in Europe that speaks Dutch.', additional_kwargs={}, response_metadata={}), HumanMessage(content='German', additional_kwargs={}, response_metadata={}), AIMessage(content='\n\nGermany is the main country in Europe that speaks German. Austria and Switzerland also have significant populations who speak German, and it is recognized as an official language in Liechtenstein, Luxembourg, and Belgium. Parts of Italy, France, and Poland also have German-speaking communities.', additional_kwargs={}, response_metadata={}), HumanMessage(content='Irish', additional_kwargs={}, response_metadata={}), AIMessage(content='\n\nIreland is the only country in Europe where Irish (Gaeilge) is recognized as an official language and is widely spoken.', additional_kwargs={}, response_metadata={})]))

In [47]:
# Access the buffer where the chat content stored
country_request.memory.buffer.strip()

'Human: Dutch\nAI: \n\nThe Netherlands is a country in Europe that speaks Dutch.\nHuman: German\nAI: \n\nGermany is the main country in Europe that speaks German. Austria and Switzerland also have significant populations who speak German, and it is recognized as an official language in Liechtenstein, Luxembourg, and Belgium. Parts of Italy, France, and Poland also have German-speaking communities.\nHuman: Irish\nAI: \n\nIreland is the only country in Europe where Irish (Gaeilge) is recognized as an official language and is widely spoken.'

Below you can see the conversation taken place between the *User* and *AI*.

In [48]:
print(country_request.memory.buffer)

Human: Dutch
AI: 

The Netherlands is a country in Europe that speaks Dutch.
Human: German
AI: 

Germany is the main country in Europe that speaks German. Austria and Switzerland also have significant populations who speak German, and it is recognized as an official language in Liechtenstein, Luxembourg, and Belgium. Parts of Italy, France, and Poland also have German-speaking communities.
Human: Irish
AI: 

Ireland is the only country in Europe where Irish (Gaeilge) is recognized as an official language and is widely spoken.


**ConversationChain** in LangChain builds on this concept of memory. It uses a memory module to *store* and *manage* the conversational context explicitly, enabling the model to "remember" previous exchanges within the same interaction.

While ChatGPT inherently maintains context within its session, a ConversationChain extends this functionality by making memory handling explicit and customizable. For example:

- **BufferMemory** stores all conversation history (like ChatGPT's session context).
- **SummaryMemory** summarizes past interactions into a concise format to save space for longer conversations.
- **EntityMemory** tracks specific entities mentioned in the conversation.
These mechanisms make ConversationChain a practical tool for building applications where maintaining dynamic context is crucial, such as customer support or dialogue systems.

According to [medium.com](https://medium.com/@RSK2327/breaking-down-langchain-chatopenai-and-conversationchain-03565f421f78), ConversationChain is a subclass of LLMChain thats meant to be used specifically for chat agent applications. However, most of the functions being used are actually being implemented in the LLMChain and its parent, Chain, class. So we can reproduce the same results by just using the LLMChain class. Yet using using ConversationChain recommended as it implements certain checks to ensure the workflow doesnt break

In [13]:
from langchain.chains import ConversationChain

In [16]:
conv1 = ConversationChain(llm=client)
# conv1

In [17]:
# print(conv1)

In [63]:
conv1.prompt, conv1.prompt.template

(PromptTemplate(input_variables=['history', 'input'], input_types={}, partial_variables={}, template='The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.\n\nCurrent conversation:\n{history}\nHuman: {input}\nAI:'),
 'The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.\n\nCurrent conversation:\n{history}\nHuman: {input}\nAI:')

In [64]:
print(conv1.prompt.template)

The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
{history}
Human: {input}
AI:


**BINGO!** Conversation Chain is created and ready to use. Lwt us make a few requests!

In [65]:
conv1.run("When was Abraham Lincoln born?")

' Abraham Lincoln was born on February 12, 1809 in a one-room log cabin in Hardin County, Kentucky. His parents were Thomas Lincoln and Nancy Hanks Lincoln. He was the second child of the couple, with his older sister Sarah being two years older.'

In [67]:
conv1.run("In a short answer, what is the capital of China")

' The capital of China is Beijing.'

In [68]:
conv1.run("In a single word, which is bigger, Beijing or London?")

' In a single word, London is bigger than Beijing.'

In [69]:
conv1.run("What was his full name?")

" Abraham Lincoln's full name was actually Abraham Lincoln, as he did not have a middle name. His parents were reportedly illiterate and did not know how to properly register his birth, so he was simply given his first and last name."

In [75]:
print(conv1.memory.buffer)

Human: When was Abraham Lincoln born?
AI:  Abraham Lincoln was born on February 12, 1809 in a one-room log cabin in Hardin County, Kentucky. His parents were Thomas Lincoln and Nancy Hanks Lincoln. He was the second child of the couple, with his older sister Sarah being two years older.
Human: Briefly, what color is this piece of text?
AI:  This piece of text appears to be black. However, it is important to note that the color of text can vary depending on the typeface, font size, and background color it is displayed on. For example, if this text was enlarged or displayed on a white background, it may appear slightly different shades of gray.
Human: In a short answer, what is the capital of China
AI:  The capital of China is Beijing.
Human: In a single word, which is bigger, Beijing or London?
AI:  In a single word, London is bigger than Beijing.
Human: What was his full name?
AI:  Abraham Lincoln's full name was actually Abraham Lincoln, as he did not have a middle name. His parents w

**ConversationBufferWindowMemory** is a specific type of memory module in LangChain that keeps track of a fixed-size sliding window of the most recent interactions in a conversation. It doesn't store the entire history like ConversationBufferMemory, but rather focuses on the last n messages.

Key Characteristics:
- Windowed Context: It remembers only a limited number of recent exchanges (e.g., the last 5 turns).
- Efficiency: By limiting the stored context, it avoids memory overload and keeps the computational load manageable, especially for long conversations.
Dynamic Updates: As new messages are added, older ones fall out of the window.

Use Case:
- This memory type is particularly useful in scenarios where:
- Short-Term Relevance: Only recent interactions matter for the conversation.
Efficiency: You need to save resources while maintaining relevant context.

In [81]:
from langchain.memory import ConversationBufferWindowMemory

In [85]:
# Initialize the memory with a window size of 3 (remembers the last 3 interactions)
memory2 = ConversationBufferWindowMemory(k=1)

In [86]:
conv2 = ConversationChain(llm=OpenAI(openai_api_key=my_openaiapi_key, temperature=1.0), 
                          memory=memory2)

In [88]:
conv2.run("In a single sentence, what is the capital of Iran?")

' The capital of Iran is Tehran, a bustling metropolis with a rich history and a population of over 8 million people, located in the northern part of the country at the foot of the Alborz Mountains.'

In [89]:
conv2.run("What is 2+2?")

' 2+2 is equal to 4, a basic mathematical operation that involves adding two numbers together.'

In [90]:
conv2.run("What is the closest city to it?")

' I would need more context to answer that question accurately. What is the "it" you are referring to? Is it a specific location or a general area?'

It is clearly noticeable that the model fails to provide a reliable answer since it has no clue what "it" refers to. We have set `k=1` and therefore the longest it can look back is for 1 single message. In other words, it can only remember one last response. To explore further about memory, you can read [LangChain Documentation](https://python.langchain.com/v0.1/docs/modules/memory/).

Hope you enjoyed the brief tutorial!